## Neural Networks
### In-class practise: Systematic data augmentation for images

by Stefan Heinrich
Based on sec 13.1 of http://d2l.ai/chapter_computer-vision/image-augmentation.html and further adaptations by Kevin Murphy in https://github.com/probml/probml-notebooks/blob/main/notebooks-d2l/image_augmentation_torch.ipynb

We illustrate some simple data augmentation methods for images.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(seed=1)
import math

import torch
from torch import nn
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms

!mkdir figures # for saving plots

!wget https://raw.githubusercontent.com/d2l-ai/d2l-en/master/d2l/torch.py -q -O d2l.py
import d2l.torch as d2l

In [3]:
d2l.set_figsize()
#img = d2l.Image.open('../img/cat1.jpg')

#url = 'https://github.com/d2l-ai/d2l-en/blob/master/img/cat1.jpg?raw=true'
url = 'https://github.com/probml/probml-notebooks/blob/main/images/cat_dog.jpg?raw=true'
fname = 'img.jpg'
!wget $url -q -O $fname


img = d2l.Image.open(fname)

d2l.plt.imshow(img);

zsh:1: no matches found: https://github.com/probml/probml-notebooks/blob/main/images/cat_dog.jpg?raw=true


FileNotFoundError: [Errno 2] No such file or directory: 'img.jpg'

In [ ]:
def show_images(imgs, num_rows, num_cols, titles=None, scale=1.5):
    """Plot a list of images."""
    figsize = (num_cols * scale, num_rows * scale)
    fig, axes = d2l.plt.subplots(num_rows, num_cols, figsize=figsize)
    axes = axes.flatten()
    for i, (ax, img) in enumerate(zip(axes, imgs)):
        if torch.is_tensor(img):
            # Tensor Image
            ax.imshow(img.numpy())
        else:
            # PIL Image
            ax.imshow(img)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        if titles:
            ax.set_title(titles[i])
    plt.tight_layout()
    return fig, axes

To visualize an image augmentation, which may be stochastic, we apply it multiple times to an image.


In [ ]:

def apply(img, aug, num_rows=1, num_cols=4, scale=2):
    Y = [aug(img) for _ in range(num_rows * num_cols)]
    fig, axes = show_images(Y, num_rows, num_cols, scale=scale)
    return fig, axes

# Flipping

In [ ]:
apply(img, torchvision.transforms.RandomHorizontalFlip())

In [ ]:
apply(img, torchvision.transforms.RandomVerticalFlip())

# Crop and resize

Below, we randomly crop a region with an area of 10% to 100% of the original area, and the ratio of width to height of the region is randomly selected from between 0.5 and 2. Then, the width and height of the region are both scaled to 200 pixels.


In [ ]:
shape_aug = torchvision.transforms.RandomResizedCrop(
    (200, 200), scale=(0.1, 1), ratio=(0.5, 2))
apply(img, shape_aug)

In [ ]:
shape_aug = torchvision.transforms.RandomResizedCrop(
    (200, 200), scale=(0.5, 1), ratio=(1, 1))
fig, axes = apply(img, shape_aug)
fig.savefig('dog_cat_augment.png')

# Changing color

We can change brightness, contrast, saturation and hue.
First we change brightness, from 1-0.5=0.5 times less to 1+0.5=1.5 times more.

In [ ]:
apply(
    img,
    torchvision.transforms.ColorJitter(brightness=0.5, contrast=0,
                                       saturation=0, hue=0))

Now we change hue.

In [ ]:
apply(
    img,
    torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0,
                                       hue=0.5))

Now we change saturation.

In [ ]:
apply(
    img,
    torchvision.transforms.ColorJitter(brightness=0, contrast=0, saturation=0.5,
                                       hue=0))

Now we change contrast.

In [ ]:
apply(
    img,
    torchvision.transforms.ColorJitter(brightness=0, contrast=0.5, saturation=0,
                                       hue=0))

Now we change all of them.

In [ ]:
color_aug = torchvision.transforms.ColorJitter(brightness=0.5, contrast=0.5,
                                               saturation=0.5, hue=0.5)
apply(img, color_aug)

# Combining multiple augmentations in a pipeline

In [ ]:
augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(), color_aug, shape_aug])
apply(img, augs)

In [ ]:
augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(), shape_aug])
fig, axes = apply(img, augs)
fig.savefig('dog_cat_augment2.png')

# Using augmentations in a dataloader

We illustrate how we can transform training and test images from FashionMNIST.

In [ ]:
all_images = torchvision.datasets.FashionMNIST(train=True, root="../data",
                                          download=True)
d2l.show_images([all_images[i][0] for i in range(32)], 4, 8, scale=0.8);

In [ ]:
train_augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor()])

test_augs = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()])

In [ ]:
batch_size = 32
dataset = torchvision.datasets.FashionMNIST(root="../data", train=True,
                                       transform=train_augs, download=True)
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True,
    num_workers=d2l.get_dataloader_workers())

In [ ]:
NO_images = batch_size

def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(dataloader)
images, label = next(dataiter)

imshow(torchvision.utils.make_grid(images))

# print images
print("This shows the next batch of images in the training set")
imshow(torchvision.utils.make_grid(images))